<a href="https://colab.research.google.com/github/doubi-xu/Political-Ideology-and-Customer-Feedback-Behaviors/blob/main/Feedback_DiagnosticValue_SentenceTopic_removeEng_ACSI06102025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### STEP 0: Install Dependencies ###
!pip install pandas pyreadstat bertopic umap-learn hdbscan nltk -q
!pip install sentence-transformers -q
!pip install langdetect

### STEP 1: Mount Google Drive ###
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.7/617.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
### STEP 2: Load and Preprocess Data ###
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

file_path = '/content/drive/My Drive/ACSIFeedback/ACSI04042025.dta'
df = pd.read_stata(file_path)

# Combine VBEST and VWORST into a single review field
df['reviewtext_cleaned'] = df['VBEST'].fillna('') + '. ' + df['VWORST'].fillna('')
df['review_length'] = df['reviewtext_cleaned'].apply(lambda x: len(str(x).split()))

print(f"✅ Loaded {len(df):,} total reviews.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


✅ Loaded 11,510 total reviews.


<ipython-input-2-1923265377>:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['reviewtext_cleaned'] = df['VBEST'].fillna('') + '. ' + df['VWORST'].fillna('')
<ipython-input-2-1923265377>:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['review_length'] = df['reviewtext_cleaned'].apply(lambda x: len(str(x).split()))


In [ ]:
### STEP 3: Tokenize into Sentences ###
df['sentences'] = df['reviewtext_cleaned'].apply(sent_tokenize)
df_sentences = df[['sentences']].explode('sentences').reset_index()
df_sentences.rename(columns={'index': 'doc_id', 'sentences': 'sentence'}, inplace=True)
df_sentences['sentence'] = df_sentences['sentence'].astype(str).str.strip()
df_sentences = df_sentences[df_sentences['sentence'].str.len() > 0]

print(f"📝 Tokenized into {len(df_sentences):,} sentences from {df['reviewtext_cleaned'].notnull().sum():,} reviews.")

📝 Tokenized into 23,299 sentences from 11,510 reviews.


<ipython-input-3-140496632>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['sentences'] = df['reviewtext_cleaned'].apply(sent_tokenize)


In [ ]:
### STEP 4: Fit BERTopic on Sentences ###
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
topic_model = BERTopic(embedding_model=embedding_model,
                       calculate_probabilities=True,
                       verbose=True)

sentences = df_sentences['sentence'].tolist()
topics, probs = topic_model.fit_transform(sentences)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2025-06-12 01:38:36,288 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/729 [00:00<?, ?it/s]

2025-06-12 01:38:42,041 - BERTopic - Embedding - Completed ✓
2025-06-12 01:38:42,042 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-06-12 01:39:15,682 - BERTopic - Dimensionality - Completed ✓
2025-06-12 01:39:15,684 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-12 01:45:37,372 - BERTopic - Cluster - Completed ✓
2025-06-12 01:45:37,383 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-06-12 01:45:37,934 - BERTopic - Representation - Completed ✓


In [ ]:
### STEP 5: Export Topic Overview ###
topic_info = topic_model.get_topic_info()
topic_info.to_csv("/content/drive/My Drive/ACSIFeedback/acsi_topic_overview_sentence_noEng_F.csv", index=False)
print("📄 Sentence-level topic overview saved — ready for manual labeling.")

📄 Sentence-level topic overview saved — ready for manual labeling.


In [ ]:
topic_info.head()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4690,-1_and_they_that_customers,"[and, they, that, customers, have, their, inte...","[always there when I need them., If only one ..."
1,0,973,0_nothing_nothings_aeesome_man,"[nothing, nothings, aeesome, man, wrong, produ...","[nothing, nothing, Nothing]"
2,1,285,1_lower_prices_pricing_slightly,"[lower, prices, pricing, slightly, ala, course...","[Lower prices, Lower prices, lower prices]"
3,2,223,2_prices_pricing_tags_tha,"[prices, pricing, tags, tha, ideal, promised, ...","[Prices., Prices., prices.]"
4,3,221,3_nothing_so__,"[nothing, so, , , , , , , , ]","[Nothing.., nothing., Nothing.]"


7/500 disagreement

In [ ]:
### STEP 6: Load Formative Topic Labels and Score Sentences ###
# Load the manually coded topic file (make sure it's saved as CSV with columns: Topic, formative)
coded_topic_path = "/content/drive/My Drive/ACSIFeedback/acsi_topic_overview_sentence_noEng_F_coded.csv"
coded_df = pd.read_csv(coded_topic_path)

# Extract list of formative topic IDs
formative_topics = coded_df[coded_df["Formative"] == 1]["Topic"].tolist()
print(f"✅ Loaded {len(formative_topics)} formative topics: {formative_topics}")

# Compute sentence-level formative topic score
df_sentences["formative_score"] = [
    sum(p[i] for i in formative_topics if i < len(p)) if p is not None else 0
    for p in probs
]

# Aggregate to document level
sentence_sum = df_sentences.groupby('doc_id')['formative_score'].sum()
sentence_count = df_sentences.groupby('doc_id')['sentence'].count()
avg_sentence_count = sentence_count.mean()

df_sentences["sentence_word_count"] = df_sentences["sentence"].apply(lambda x: len(str(x).split()))
avg_sentence_length = df_sentences.groupby("doc_id")["sentence_word_count"].mean()


✅ Loaded 400 formative topics: [-1, 1, 2, 6, 8, 9, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 33, 34, 36, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 63, 64, 65, 66, 68, 69, 71, 72, 73, 74, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 91, 92, 93, 95, 96, 97, 98, 99, 100, 102, 103, 104, 105, 106, 107, 108, 109, 110, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 123, 124, 126, 127, 128, 129, 130, 131, 132, 133, 134, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 162, 163, 165, 166, 167, 168, 169, 170, 172, 173, 174, 176, 177, 178, 179, 180, 181, 183, 184, 185, 188, 191, 192, 193, 195, 196, 197, 199, 200, 201, 202, 203, 204, 205, 207, 208, 209, 210, 211, 213, 214, 215, 216, 218, 219, 222, 223, 224, 225, 226, 227, 228, 229, 231, 232, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 246, 247, 248, 249, 250, 251, 253, 254, 255, 256, 257, 258,

In [ ]:
### STEP 7: Summary Statistics for Sentence-Level Modeling ###

# Basic stats for sentence-level formative scores
print("📊 Summary: Sentence-Level Formative Scores")
print(df_sentences["formative_score"].describe())

# Aggregate diagnostic value per review
sentence_sum = df_sentences.groupby("doc_id")["formative_score"].sum()
print("\n📊 Summary: Total Diagnostic Value per Review (Sum of Sentences)")
print(sentence_sum.describe())

# Number of sentences per review
sentence_count = df_sentences.groupby("doc_id")["sentence"].count()
print("\n📊 Summary: Sentence Count per Review")
print(sentence_count.describe())


📊 Summary: Sentence-Level Formative Scores
count     2.329900e+04
mean      5.018866e-01
std       3.720668e-01
min      3.815601e-305
25%       1.240982e-01
50%       4.800000e-01
75%       9.041519e-01
max       1.419208e+00
Name: formative_score, dtype: float64

📊 Summary: Total Diagnostic Value per Review (Sum of Sentences)
count     1.151000e+04
mean      1.015939e+00
std       6.511866e-01
min      4.587199e-305
25%       5.181448e-01
50%       1.000000e+00
75%       1.453106e+00
max       9.527231e+00
Name: formative_score, dtype: float64

📊 Summary: Sentence Count per Review
count    11510.000000
mean         2.024240
std          0.675286
min          1.000000
25%          2.000000
50%          2.000000
75%          2.000000
max         13.000000
Name: sentence, dtype: float64


In [ ]:
### STEP 8: Compute Two Diagnostic Value Metrics ###
# Create final diagnostic dataframe
diagnostic_df = pd.DataFrame({
    "diagnostic_value_sum": sentence_sum,
    "avg_sentence_length": avg_sentence_length,
    "sentence_count": sentence_count
})
diagnostic_df["diagnostic_value_scaled"] = diagnostic_df["diagnostic_value_sum"] * diagnostic_df["avg_sentence_length"]


In [ ]:
print(diagnostic_df.head())

        diagnostic_value_sum  avg_sentence_length  sentence_count  \
doc_id                                                              
0                   1.991189                  2.5               2   
1                   1.436600                  1.5               2   
2                   0.408225                  6.0               1   
3                   0.807633                  1.5               2   
4                   0.121317                 16.0               1   

        diagnostic_value_scaled  
doc_id                           
0                      4.977973  
1                      2.154900  
2                      2.449350  
3                      1.211449  
4                      1.941069  


In [ ]:
### STEP 9: Merge Back and Save ###
df = df.join(diagnostic_df, how="left")

output_path = "/content/ACSI04042025_NewElaborationnoEng0611F.csv"
df.to_csv(output_path, index=False)
print(f"✅ File saved: {output_path}")

✅ File saved: /content/ACSI04042025_NewElaborationnoEng0611F.csv
